# Correlation of Latencies and Solarmagnetic Storms

In [ ]:
import pycountry
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from calendar import monthrange
from calendar import day_name
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from math import inf

def country_name(country_iso):
    return pycountry.countries.get(alpha_2=country_iso).name

def filter_for(df, name, value):
    return df.loc[df[name] == value]

def filter_for_unequal(df, name, value):
    return df.loc[df[name] != value]

def filter_for_range(df, name, lower_bound, upper_bound):
    return df[(df[name] >= lower_bound) & (df[name] < upper_bound)]

def days_in_month(year, month):
    return monthrange(year, month)[1]

def first_weekday_of_month(year, month):
    return monthrange(year, month)[0]

def date_to_weekday(day, month, year):
    date = datetime(year, month, day)
    return day_name[date.weekday()]

from kpdownload import getKpindex

In [ ]:
country = 'US'
df = pd.read_parquet('./parquet/tls_data.parquet').join(pd.read_parquet('./parquet/ripe_atlas_probe_data.parquet').set_index('id'), on='prb_id')
df = filter_for(df, 'country', country)

df

In [ ]:
one_day = timedelta(days=1)
three_hours = timedelta(hours=3)
data = []

interval = one_day

current = datetime(2022, 1, 1)
max = datetime.fromtimestamp(np.max(df['timestamp'].tolist()))

while current < max:
    median_rt = np.median(filter_for_range(df, 'timestamp', current.timestamp(), (current + interval).timestamp())['rt'])
    average_kp = np.mean(getKpindex(current.isoformat() + 'Z', (current + interval).isoformat() + 'Z', 'Kp')[1])
    data.append( (median_rt, average_kp) )

    current += interval

solar_df = pd.DataFrame(data, columns=['Latency', 'Kp Index'])
solar_df

In [ ]:
pearson_correlation = solar_df['Latency'].corr(solar_df['Kp Index'], method='pearson')
kendall_correlation = solar_df['Latency'].corr(solar_df['Kp Index'], method='kendall')
spearman_correlation = solar_df['Latency'].corr(solar_df['Kp Index'], method='spearman')

print(np.round(pearson_correlation, 2))
print(np.round(kendall_correlation, 2))
print(np.round(spearman_correlation, 2))